# Jupyter Notebook for converting the Go Pass Data


## 1.0 Load in modules

We are using pandas and the numpy libraries to read and process the data.

Then we will use the `rapidfuzz` library to caclulate close matches to the strings so that we can join the information.

Using other popular fuzzy-string matching libraries like `fuzzball` takes over 3 hours to process as the alogrithim used takes the number of records to the `n`th power!! (i.e. comparing 1000 records processes 1000^1000!!!) `rapidfuzz` has a better algorithm that trims this town to 3 - 4 minutes!


In [1]:
import pandas as pd
import numpy as np
from datetime import datetime
from rapidfuzz import process, utils as fuzz_utils

GOOGLE_SHEET_URL = 'https://docs.google.com/spreadsheets/d/e/2PACX-1vSWbwrsqF-c---4lfw0LZWymd-f8sy8sLYkXgzh0OyeGATWwrvv7V1Mq5BcApn7F_-WYKP1KXy5shKw/pub?gid=376323488&single=true&output=csv'

## 2.0 Format Go Pass Data

We will be extracting the following fields:

- District
- School
- Address
- City
- Participating

We create a simplfied field called `name` based on the `street address`, `school name`, and `district`. This field is formatted to be all lowercase with spaces replaced with `-` to make matching easier.

In [2]:
go_pass_schools = pd.read_csv(GOOGLE_SHEET_URL,
    usecols={'district_x','original_name_x','address_x','city_x','phone','email'})
go_pass_schools.columns = ["district","name","address","city","phone","email"]

go_pass_schools = go_pass_schools.fillna('')
go_pass_schools['original_name'] = go_pass_schools['name']
simple_district = go_pass_schools['district'].str.lower().str.replace(" ","-").str.replace(".","").str.replace('"','')
# go_pass_schools['name'] = go_pass_schools['name'].str.lower().str.replace(" ","-").str.replace(".","").str.replace('"','')+"-"+simple_district
simple_address = go_pass_schools['address'].str.lower().str.replace(" ","-").str.replace(".","").str.replace('"','')
simple_name = go_pass_schools['name'].str.lower().str.replace(" ","-").str.replace(".","").str.replace('"','')
go_pass_schools.loc[(go_pass_schools['name'].str.len() > 1),'name'] = simple_name+"-"+simple_address
go_pass_schools['participating'] = True
go_pass_schools



KeyboardInterrupt: 

### 2.1 Format California Schools data

We create a simplfied field called `simple name` based on the `street address`, `school name`, and `district`. This field is formatted to be all lowercase with spaces replaced with `-` to make matching easier.

#### Source for California schools data

https://www.cde.ca.gov/SchoolDirectory/ExportSelect?simpleSearch=N&address=&city=&counties=&districts=&cdscode=&charter=&magnet=&name=&nps=&search=2&zip=&yearround=&status=1%2C2&types=&order=1&multilingual=&qsc=3549&qdc=3549

August 2022


In [ ]:
california_schools_data = "../data/TapData.csv"

california_schools = pd.read_csv(california_schools_data)
    # usecols={'school','district','street','city','state',"zip"})
# california_schools.columns = ["school","district","address","city","state","zip"]

la_schools_df = california_schools.fillna('')
# la_schools_df

la_schools_df['original_name'] = la_schools_df['school']
name = la_schools_df['school'].replace({'\([a-zA-Z\s\.\-\/]*\(*[a-zA-Z\s\.\-\/]*\)*[a-zA-Z\s\.\-\/]*\)$'},'',regex=True)
la_schools_df['name'] = name
# la_schools_df.loc[la_schools_df['district'] == "Los Angeles Unified", 'district'] = "LAUSD"
simple_district = la_schools_df['district'].str.lower().str.replace(" ","-").str.replace(".","").str.replace('"','')
simple_address = la_schools_df['street'].str.lower().str.replace(" ","-").str.replace(".","").str.replace('"','')
simple_name = la_schools_df['name'].str.lower().str.replace(" ","-").str.replace(".","").str.replace('"','')+"-"
la_schools_df['simple_name'] = simple_name+"-"+simple_address
la_schools_df['simple_name'] = la_schools_df['simple_name'].str.replace("---","-")


la_schools_df

<ipython-input-8-085443f20ffb>:14: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  simple_district = la_schools_df['district'].str.lower().str.replace(" ","-").str.replace(".","").str.replace('"','')
<ipython-input-8-085443f20ffb>:15: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  simple_address = la_schools_df['street'].str.lower().str.replace(" ","-").str.replace(".","").str.replace('"','')
<ipython-input-8-085443f20ffb>:16: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  simple_name = la_schools_df['name'].str.lower().str.

,rowid,school,district,street,city,state,zip,original_name,name,simple_name
0,1,Palms Elementary (ABC Unified),ABC Unified,12445 East 207th St.,Lakewood,CA,90715-1619,Palms Elementary (ABC Unified),Palms Elementary,palms-elementary-abc-unified
1,2,Ross (Faye) Middle (ABC Unified),ABC Unified,17707 Elaine Ave.,Artesia,CA,90701-4018,Ross (Faye) Middle (ABC Unified),Ross (Faye) Middle,ross-(faye)-middle-abc-unified
2,3,Stowers(Cecil B.) Elementary (ABC Unified),ABC Unified,13350 Beach St.,Cerritos,CA,90703-1331,Stowers(Cecil B.) Elementary (ABC Unified),Stowers(Cecil B.) Elementary,stowers(cecil-b)-elementary-abc-unified
3,4,Tetzlaff (Martin B.) Middle (ABC Unified),ABC Unified,12351 East Del Amo Blvd.,Cerritos,CA,90703-7635,Tetzlaff (Martin B.) Middle (ABC Unified),Tetzlaff (Martin B.) Middle,tetzlaff-(martin-b)-middle-abc-unified
4,5,Tracy (Wilbur) High (Continuation) (ABC Unified),ABC Unified,12222 Cuesta Dr.,Cerritos,CA,90703-2801,Tracy (Wilbur) High (Continuation) (ABC Unified),Tracy (Wilbur) High (Continuation),tracy-(wilbur)-high-(continuation)-abc-unified
...,...,...,...,...,...,...,...,...,...,...
3146,3147,YouthBuild - Norwalk (YouthBuild Charter Schoo...,YouthBuild Charter Schools of Los Angeles,12124 Front St.,Norwalk,CA,90650,YouthBuild - Norwalk (YouthBuild Charter Schoo...,YouthBuild - Norwalk,youthbuild-norwalk-youthbuild-charter-schools-...
3147,3148,YouthBuild - Palmdale (YouthBuild Charter Scho...,YouthBuild Charter Schools of Los Angeles,38626 9th St. East,Palmdale,CA,93550,YouthBuild - Palmdale (YouthBuild Charter Scho...,YouthBuild - Palmdale,youthbuild-palmdale-youthbuild-charter-schools...
3148,3149,YouthBuild - Pomona (YouthBuild Charter School...,YouthBuild Charter Schools of Los Angeles,305 E Arrow Hwy.,Pomona,CA,91767,YouthBuild - Pomona (YouthBuild Charter School...,YouthBuild - Pomona,youthbuild-pomona-youthbuild-charter-schools-o...
3149,3150,YouthBuild - South LA (YouthBuild Charter Scho...,YouthBuild Charter Schools of Los Angeles,400 West Washington Blvd.,Los Angeles,CA,90015,YouthBuild - South LA (YouthBuild Charter Scho...,YouthBuild - South LA,youthbuild-south-la-youthbuild-charter-schools...


In [ ]:
# unique_schools = go_pass_schools.drop_duplicates(subset=['original_name'])
# print(go_pass_schools.shape)
# print(unique_schools.shape)

# difference_in_dupes = go_pass_schools.shape[0] - unique_schools.shape[0]
# # these are the number of records with the same names
# print("Number of records with same names: \n "+str(difference_in_dupes))

nina_q = la_schools_df.loc[la_schools_df['simple_name'].str.contains('alliance')]
# nina_q = la_schools_df.query('simple_name.str.contains("alliance")')
# go_pass_schools.loc[]
for a in nina_q['simple_name']:
    print(a)


alliance-leichtman-levine-family-foundation-environmental-science-high-alliance-college-ready-public-schools-district
alliance-marc-&-eva-stern-math-and-science-alliance-college-ready-public-schools-district
alliance-margaret-m-bloomfield-technology-academy-high-alliance-college-ready-public-schools-district
alliance-marine-innovation-and-technology-6-12-complex-alliance-college-ready-public-schools-district
alliance-morgan-mckinzie-high-alliance-college-ready-public-schools-district
alliance-ouchi-o'donovan-6-12-complex-alliance-college-ready-public-schools-district
alliance-patti-and-peter-neuwirth-leadership-academy-alliance-college-ready-public-schools-district
alliance-piera-barbaglia-shaheen-health-services-academy-alliance-college-ready-public-schools-district
alliance-renee-and-meyer-luskin-academy-high-alliance-college-ready-public-schools-district
alliance-susan-and-eric-smidt-technology-high-alliance-college-ready-public-schools-district
alliance-ted-k-tajima-high-alliance-c

## 3.0 Joining the `Original dataset` to the `California dataset`

Here we do a blanket `left` merge where the original records get data added to it.

If we want to keep the California data, then we need to switch this merge type to `inner` or `right`.

#### See the Pandas merge documentation for more information:

https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.merge.html

In [ ]:
def fuzzy_merge(baseFrame, compareFrame, baseKey, compareKey, threshold=86, limit=1, how='left'):
    s_mapping = {x: fuzz_utils.default_process(x) for x in compareFrame[compareKey]}

    m1 = baseFrame[baseKey].apply(lambda x: process.extract(
      fuzz_utils.default_process(x), s_mapping, limit=limit, score_cutoff=threshold, processor=None
    ))
    baseFrame['Match'] = m1

    m2 = baseFrame['Match'].apply(lambda x: ', '.join(i[2] for i in x))
    baseFrame['name'] = m2.replace("",np.nan)
    # baseFrame['school'] = m2['school']
    return baseFrame.merge(compareFrame, left_on='name', right_on=compareKey, how=how)

merged_df = fuzzy_merge(go_pass_schools, la_schools_df, 'name', 'simple_name',how='outer')
# merged_df = fuzzy_merge(go_pass_schools, la_schools_df, 'original_name', 'name',how='left')
# merged_df = fuzzy_merge(la_schools_df, go_pass_schools, 'name', 'original_name',how='right')
merged_df


KeyboardInterrupt: 

## 4.0 Cleaning up of Merged Data

The steps are as follows:
1. Assign the full name the original name and the district
   - Note: you can also use the California district names instead by uncommenting the `Altenative` line
2. Process alternative method of naming where district names are only added if there are duplicates
3. Set the final `phone` column to default to the original data's `telephone` column
4. Get the match score from rapidfuzz

In [ ]:
merged_df['rowid'].fillna(0, inplace=True)
merged_df['rowid'] = merged_df['rowid'].astype('int64')

In [ ]:
# 1. Default name processing
# merged_df['full_name'] = merged_df['original_name_x'] + ' (' + merged_df['district_x'] + ')'
merged_df.loc[merged_df['rowid'] == 0, 'school'] = merged_df['original_name_x'] + ' (' + merged_df['district_x'] + ')'
merged_df.loc[merged_df['rowid'] == 0, 'district'] = merged_df['district_x']
# 1b. alternative approach: uncomment below to use the California district names instead
# merged_df['full_name']  = merged_df['original_name_x'] + ' (' + merged_df['district_y'] + ')'

# 2. Alternative school name field processing
# Flag duplicated records
merged_df['duped'] = merged_df.duplicated(['original_name_x'],keep=False)

# `full_name_some` is the option where there are district names for only duplicated records.
# merged_df.loc[merged_df['duped'] == False, 'full_name_some'] = merged_df['original_name_x']
# merged_df.loc[merged_df['duped'] == True, 'full_name_some'] = merged_df['original_name_x'] + ' (' + merged_df['district_y'] + ')'

# 3. Phone number processing
# merged_df['phone'] = merged_df['telephone']

# if there is less than 4 characters in that column then set it to the California's phone numbers i.e. 'default phone' column
# merged_df.loc[merged_df['phone'].str.len() < 4, 'phone'] = merged_df['default_phone']

# 4. Match score processing
merged_df['score'] = merged_df['Match'].astype('string').str.split(",").str[1]

merged_df

,district_x,name_x,address,city_x,phone,email,original_name_x,participating,Match,rowid,...,street,city_y,state,zip,original_name_y,name_y,simple_name,district,duped,score
0,Centinela Valley,hawthorne-high-centinela-valley-union-high,4859 West El Segundo Blvd.,Hawthorne,(310) 263-4400,No Data,Hawthorne High,True,"[(hawthorne high centinela valley union high, ...",306,...,4859 West El Segundo Blvd.,Hawthorne,CA,90250-4204,Hawthorne High (Centinela Valley Union High),Hawthorne High,hawthorne-high-centinela-valley-union-high,NaN,False,95.0
1,Centinela Valley,lawndale-high-centinela-valley-union-high,14901 S. Inglewood Avenue,Lawndale,(310) 263-3100,No Data,Lawndale High,True,"[(lawndale high centinela valley union high, 9...",307,...,14901 South Inglewood Ave.,Lawndale,CA,90260-1251,Lawndale High (Centinela Valley Union High),Lawndale High,lawndale-high-centinela-valley-union-high,NaN,False,95.0
2,Centinela Valley,leuzinger-high-centinela-valley-union-high,4118 West Rosecrans Ave.,Lawndale,(310) 263-2200,No Data,Leuzinger High,True,"[(leuzinger high centinela valley union high, ...",308,...,4118 West Rosecrans Ave.,Lawndale,CA,90260-1601,Leuzinger High (Centinela Valley Union High),Leuzinger High,leuzinger-high-centinela-valley-union-high,NaN,False,95.0
3,Centinela Valley,r-k-lloyde-high-centinela-valley-union-high,4951 Marine Ave.,Lawndale,(310) 263-3264,No Data,R. K. Lloyde High,True,"[(r k lloyde high centinela valley union high,...",310,...,4951 Marine Ave.,Lawndale,CA,90260-1251,R. K. Lloyde High (Centinela Valley Union High),R. K. Lloyde High,r-k-lloyde-high-centinela-valley-union-high,NaN,False,95.0
4,Charter,whitney-(gretchen)-high-abc-unified,"461 9th Street, San Pedro",San Pedro,(310) 221-0430,,Alliance Alice M. Baxter College-Ready High Sc...,True,"[(whitney gretchen high abc unified, 85.5, w...",7,...,16800 Shoemaker Ave.,Cerritos,CA,90703-1244,Whitney (Gretchen) High (ABC Unified),Whitney (Gretchen) High,whitney-(gretchen)-high-abc-unified,NaN,False,85.5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4271,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3147,...,12124 Front St.,Norwalk,CA,90650,YouthBuild - Norwalk (YouthBuild Charter Schoo...,YouthBuild - Norwalk,youthbuild-norwalk-youthbuild-charter-schools-...,NaN,True,NaN
4272,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3148,...,38626 9th St. East,Palmdale,CA,93550,YouthBuild - Palmdale (YouthBuild Charter Scho...,YouthBuild - Palmdale,youthbuild-palmdale-youthbuild-charter-schools...,NaN,True,NaN
4273,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3149,...,305 E Arrow Hwy.,Pomona,CA,91767,YouthBuild - Pomona (YouthBuild Charter School...,YouthBuild - Pomona,youthbuild-pomona-youthbuild-charter-schools-o...,NaN,True,NaN
4274,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3150,...,400 West Washington Blvd.,Los Angeles,CA,90015,YouthBuild - South LA (YouthBuild Charter Scho...,YouthBuild - South LA,youthbuild-south-la-youthbuild-charter-schools...,NaN,True,NaN


## 5.0 Final Column Selection
Here we select the final data columns for our outputs, with `_x` suffixes representing the original Metro dataset and `_y` suffixes representing the California schools dataset.

- "district_x"
- "district_y"
- "original_name_x"
- "original_name_y"
- "status"
- 'closed_date'
- "full_name"
- "full_name_some"
- "participating"
- "address_x"
- "address_y"
- "city_x"
- "city_y"
- 'address_y'
- 'city_y'
- "score"
- "duped"
- 'phone'
- 'email'
- 'website'
- 'latitude'
- 'longitude'
- 'last_update'

In [ ]:
# final_columns = {
#     "full_name":"school_name",
#     "full_name_some":"school_name_with_some_districts_attached"
# }

# final_df = merged_df[["oid","district_x","district_y","original_name_x","original_name_y","status",'closed_date',
#        "full_name","full_name_some","participating","address_x","address_y","city_x","city_y","score","duped",'phone', 'email',
#        'website', 'latitude', 'longitude', 'last_update']]
final_df = merged_df

# final_df["address"] = final_df["address_x"] 


# final_df.rename(inplace=True)
# final_df.reset_index(inplace=True)
# final_df.rename(inplace=True, columns=final_columns)
final_df.reset_index(inplace=True)
# final_df.index.names = ['id']
final_df.index.names = ['id']

final_df

,index,district_x,name_x,address,city_x,phone,email,original_name_x,participating,Match,...,street,city_y,state,zip,original_name_y,name_y,simple_name,district,duped,score
id,,,,,,,,,,,,,,,,,,,,,
0,0,Centinela Valley,hawthorne-high-centinela-valley-union-high,4859 West El Segundo Blvd.,Hawthorne,(310) 263-4400,No Data,Hawthorne High,True,"[(hawthorne high centinela valley union high, ...",...,4859 West El Segundo Blvd.,Hawthorne,CA,90250-4204,Hawthorne High (Centinela Valley Union High),Hawthorne High,hawthorne-high-centinela-valley-union-high,NaN,False,95.0
1,1,Centinela Valley,lawndale-high-centinela-valley-union-high,14901 S. Inglewood Avenue,Lawndale,(310) 263-3100,No Data,Lawndale High,True,"[(lawndale high centinela valley union high, 9...",...,14901 South Inglewood Ave.,Lawndale,CA,90260-1251,Lawndale High (Centinela Valley Union High),Lawndale High,lawndale-high-centinela-valley-union-high,NaN,False,95.0
2,2,Centinela Valley,leuzinger-high-centinela-valley-union-high,4118 West Rosecrans Ave.,Lawndale,(310) 263-2200,No Data,Leuzinger High,True,"[(leuzinger high centinela valley union high, ...",...,4118 West Rosecrans Ave.,Lawndale,CA,90260-1601,Leuzinger High (Centinela Valley Union High),Leuzinger High,leuzinger-high-centinela-valley-union-high,NaN,False,95.0
3,3,Centinela Valley,r-k-lloyde-high-centinela-valley-union-high,4951 Marine Ave.,Lawndale,(310) 263-3264,No Data,R. K. Lloyde High,True,"[(r k lloyde high centinela valley union high,...",...,4951 Marine Ave.,Lawndale,CA,90260-1251,R. K. Lloyde High (Centinela Valley Union High),R. K. Lloyde High,r-k-lloyde-high-centinela-valley-union-high,NaN,False,95.0
4,4,Charter,whitney-(gretchen)-high-abc-unified,"461 9th Street, San Pedro",San Pedro,(310) 221-0430,,Alliance Alice M. Baxter College-Ready High Sc...,True,"[(whitney gretchen high abc unified, 85.5, w...",...,16800 Shoemaker Ave.,Cerritos,CA,90703-1244,Whitney (Gretchen) High (ABC Unified),Whitney (Gretchen) High,whitney-(gretchen)-high-abc-unified,NaN,False,85.5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4271,4271,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,12124 Front St.,Norwalk,CA,90650,YouthBuild - Norwalk (YouthBuild Charter Schoo...,YouthBuild - Norwalk,youthbuild-norwalk-youthbuild-charter-schools-...,NaN,True,NaN
4272,4272,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,38626 9th St. East,Palmdale,CA,93550,YouthBuild - Palmdale (YouthBuild Charter Scho...,YouthBuild - Palmdale,youthbuild-palmdale-youthbuild-charter-schools...,NaN,True,NaN
4273,4273,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,305 E Arrow Hwy.,Pomona,CA,91767,YouthBuild - Pomona (YouthBuild Charter School...,YouthBuild - Pomona,youthbuild-pomona-youthbuild-charter-schools-o...,NaN,True,NaN


In [ ]:
final_df.columns

Index(['index', 'district_x', 'name_x', 'address', 'city_x', 'phone', 'email',
       'original_name_x', 'participating', 'Match', 'rowid', 'school',
       'district_y', 'street', 'city_y', 'state', 'zip', 'original_name_y',
       'name_y', 'simple_name', 'district', 'duped', 'score'],
      dtype='object')

## 5.0 Final Output
Using today's date and the csv file extension we will output the file to the data directory.

We also split the data using `to_json` in pandas, more info here:

https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.to_json.html

In [ ]:
today = str(datetime.now().strftime("%Y-%m"))
outfile_extension = ".csv"
output_file_name = "../data/tap_data_merged_with_metro_california_qc_data_right_join_"+today+outfile_extension

final_df.to_csv(output_file_name)

#create JSON file oriented by records
output_json = "../src/data/schools_right_join.json"
json_file = final_df.to_json(orient='records',index=True) 
with open(output_json, 'w') as f:
    f.write(json_file)

NameError: name 'datetime' is not defined

In [ ]:
# Done!